In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re, pip, conda

In [2]:
for package in [sklearn,mlp,np,pd,sns,pip,conda]:
    print(re.findall("([^']*)",str(package))[2],package.__version__)

sklearn 1.0.1
matplotlib 3.4.3
numpy 1.22.4
pandas 1.3.4
seaborn 0.11.2
pip 22.2.2
conda 4.12.0


In [3]:
#pip install --upgrade scikit-learn
#conda update scikit-learn

In [35]:
#原始数据所在目录，你可以将该目录修改为你的数据集所在目录
PATH = r"D:\pythonwork\2022DACase\UserRetentionPrediction\original_data\\"

In [37]:
test = pd.read_csv(PATH + r"test-a-without-label.csv",header=0)
#test.columns = ["用户ID","(脱敏后)需要预测的留存分日期"] #中文更有利于理解，英文更有利于敲代码
test.columns = ["user_id","predict_date"]

In [38]:
test.shape #一共1w5用户，出乎意料地少

(15001, 2)

In [39]:
test.head()

,user_id,predict_date
0,10007813,205
1,10052988,210
2,10279068,200
3,10546696,216
4,10406659,183


In [6]:
launch = pd.read_csv(PATH + r"app_launch_logs.csv",header=0)
launch.columns = ["用户ID","登录类型","(脱敏后)登录APP的日期"]

In [7]:
launch.shape #包含不同用户849w+次的登录信息

(8493878, 3)

In [8]:
launch.head()

,用户ID,登录类型,(脱敏后)登录APP的日期
0,10157996,0,129
1,10139583,0,129
2,10277501,0,129
3,10099847,0,129
4,10532773,0,129


In [48]:
launch["user_id"].unique().__len__()

600000

In [14]:
playback = pd.read_csv(PATH + r"user_playback_data.csv",header=0)
playback.columns = ["用户ID","视频ID","观看时长","(脱敏后)观看日期"]

In [15]:
playback.shape #包含不同用户7100w+次浏览记录

(71046035, 4)

In [16]:
playback.head() 

,用户ID,视频ID,观看时长,(脱敏后)观看日期
0,10057286,20628283.0,2208.612,145
1,10522615,23930557.0,31.054,145
2,10494028,20173699.0,115.952,145
3,10181987,21350426.0,1.585,145
4,10439175,22946929.0,51.726,145


In [17]:
playback["观看时长"].describe() #从最大值为25134来看，观看时长的单位应该是秒

count    7.104604e+07
mean     4.054142e+02
std      8.614861e+02
min      1.000000e-03
25%      8.602000e+00
50%      6.729900e+01
75%      2.606180e+02
max      2.513487e+05
Name: 观看时长, dtype: float64

In [28]:
#train = pd.read_csv(train_PATH + r'online_train.csv',header=0)
train.head()

,user_id,date_list,date_max,date_min,date,label
0,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,128,0
0,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,129,0
0,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,130,0
0,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,131,0
0,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,132,0


- **视频观看数据的滑窗与衍生**

In [29]:
user_pb = pd.read_csv(PATH + r'user_playback_data_1.csv')
user_pb['count'] = 1

In [30]:
user_pb.head()

,user_id,video_id,play_duration,play_date,count
0,10057286,20628283.0,2208.612,145,1
1,10522615,23930557.0,31.054,145,1
2,10494028,20173699.0,115.952,145,1
3,10181987,21350426.0,1.585,145,1
4,10439175,22946929.0,51.726,145,1


In [31]:
#首先，我们只关心那些在测试集中的用户
user_pb_group = user_pb[user_pb['user_id'].isin(test['user_id'])]

In [32]:
#对每一个用户ID，计算这个用户ID在play_date下的视频播放时长之和、视频播放次数之和
user_pb_group = user_pb_group[['user_id','play_date','play_duration','count']].groupby(['user_id','play_date'],as_index=False).agg(sum)

In [33]:
user_pb_group.head()

,user_id,play_date,play_duration,count
0,10000176,162,770.829,2
1,10000176,163,16187.517,7
2,10000176,165,13584.509,9
3,10000176,166,7742.622,6
4,10000176,167,1818.905,5


In [34]:
user_pb_group.columns = ["user_id","date","play_duration","count"]

In [35]:
user_pb_group.head()

,user_id,date,play_duration,count
0,10000176,162,770.829,2
1,10000176,163,16187.517,7
2,10000176,165,13584.509,9
3,10000176,166,7742.622,6
4,10000176,167,1818.905,5


In [36]:
train.head()

,user_id,date_list,date_max,date_min,date,label
0,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,128,0
0,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,129,0
0,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,130,0
0,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,131,0
0,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,132,0


In [37]:
#我们将用户在特定日期下的视频播放时长、播放次数与原始训练集合并，采用左连接方式
#对user_pb_group来说，user_id + date被merge识别为索引了，因此merge函数会将剩下的两列play_duration和count合并到train中
train = pd.merge(train, user_pb_group, how='left')
#由于会按照user_id + date进行匹配，因此现在被放入train表单的是date当日的播放时长和播放次数数据
train.rename(columns = {'play_duration':'playtime_last'+str(0),'count':'video_count_last'+str(0)},inplace=True)

In [38]:
train.head()

,user_id,date_list,date_max,date_min,date,label,playtime_last0,video_count_last0
0,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,128,0,NaN,NaN
1,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,129,0,NaN,NaN
2,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,130,0,NaN,NaN
3,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,131,0,NaN,NaN
4,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,132,0,NaN,NaN


In [39]:
user_pb_group[user_pb_group["user_id"] == 10000176].head() #明显对于10000176这个用户来说，他/她的观看记录都是162日期之后，因此前面预测日期为128的时候，自然匹配不到相关数据

,user_id,date,play_duration,count
0,10000176,162,770.829,2
1,10000176,163,16187.517,7
2,10000176,165,13584.509,9
3,10000176,166,7742.622,6
4,10000176,167,1818.905,5


In [40]:
train[(train["user_id"] == 10000176) & (train["date"] > 160)] #从date=162开始就有了匹配数据。证明merge功能是运行正常的

,user_id,date_list,date_max,date_min,date,label,playtime_last0,video_count_last0
33,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,161,6,NaN,NaN
34,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,162,6,770.829,2.0
35,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,163,6,16187.517,7.0
36,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,164,6,NaN,NaN
37,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,165,6,13584.509,9.0
38,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,166,6,7742.622,6.0
39,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,167,6,1818.905,5.0
40,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,168,6,6.739,1.0
41,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,169,6,1128.180,1.0
42,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,170,6,1822.148,1.0


In [41]:
#对测试集我们执行同样的操作
test = pd.merge(test, user_pb_group, how='left')
test.rename(columns = {'play_duration':'playtime_last'+str(0),'count':'video_count_last'+str(0)},inplace=True)

In [42]:
#通过merge，已经整理好了date当日的视频播放情况，那要怎么把预测日期历史7天的数据都补足呢？
for i in range(7):
    #在for循环中不断增加user_pb_group中的date
    #例如，在最开始循环时，一行数据的date是162，经过7次循环，这行数据的date就会变为169，但这行数据所对应的其他列依然是162那天的数据
    #由于train中的date是不变的，因此在最后一次匹配时，train中的162匹配到的会是原本为（162-7）那天的user_pb_group的数据
    #通过这种方式让train中的date不断获得历史数据，最多获得历史7天的数据
    user_pb_group['date'] = user_pb_group['date'] + 1
    
    #在修改date后用merge匹配
    train = pd.merge(train, user_pb_group, how='left')
    train.rename(columns = {'play_duration':'playtime_last'+str(i+1),'count':'video_count_last'+str(i+1)},inplace=True)
    
    test = pd.merge(test, user_pb_group, how='left')
    test.rename(columns = {'play_duration':'playtime_last'+str(i+1),'count':'video_count_last'+str(i+1)},inplace=True)

In [43]:
train.head() #观看视频时长的相关特征就滑窗完毕了

,user_id,date_list,date_max,date_min,date,label,playtime_last0,video_count_last0,playtime_last1,video_count_last1,...,playtime_last3,video_count_last3,playtime_last4,video_count_last4,playtime_last5,video_count_last5,playtime_last6,video_count_last6,playtime_last7,video_count_last7
0,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,128,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,129,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,130,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,131,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000176,"[120, 144, 161, 162, 163, 165, 166, 167, 168, ...",185,120,132,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
#此时的特征名称为
train.columns

Index(['user_id', 'date_list', 'date_max', 'date_min', 'date', 'label',
       'playtime_last0', 'video_count_last0', 'playtime_last1',
       'video_count_last1', 'playtime_last2', 'video_count_last2',
       'playtime_last3', 'video_count_last3', 'playtime_last4',
       'video_count_last4', 'playtime_last5', 'video_count_last5',
       'playtime_last6', 'video_count_last6', 'playtime_last7',
       'video_count_last7'],
      dtype='object')

In [45]:
#某些日期用户没有视频播放行为，我们则使用0填充的方式进行填充
train = train.fillna(0)
test = test.fillna(0)

In [46]:
#将视频播放数据和视频播放次数保存一下
pb_feats = []
for i in range(8):
    pb_feats.append('playtime_last'+str(i))
    pb_feats.append('video_count_last'+str(i))

In [47]:
#构建新的视频播放表单
train[['user_id','date']+pb_feats].to_csv(train_PATH + r'online_train_pb.csv',index=False)
test[['user_id','date']+pb_feats].to_csv(train_PATH + r'online_test_pb.csv',index=False)